In [1]:
! pip install torch_geometric

In [2]:
import torch
from torch.optim import Adam
import torch_geometric
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GAE
import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges

In [3]:
dataset = Planetoid(root='./data', name="CiteSeer", transform = T.NormalizeFeatures())
data = dataset[0]

In [4]:
print(data)

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


In [5]:
data.train_mask = data.val_mask = data.test_mask = None

In [6]:
data = train_test_split_edges(data)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [7]:
print(data)

Data(x=[3327, 3703], y=[3327], val_pos_edge_index=[2, 227], test_pos_edge_index=[2, 455], train_pos_edge_index=[2, 7740], train_neg_adj_mask=[3327, 3327], val_neg_edge_index=[2, 227], test_neg_edge_index=[2, 455])


In [8]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()

        self.conv1 = GCNConv(in_channels, 2*out_channels)
        self.conv2 = GCNConv(2*out_channels, 4*out_channels)
        self.conv3 = GCNConv(4*out_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        return self.conv3(x, edge_index)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
out_channels = 2
num_features = dataset.num_features

model = GAE(GCNEncoder(num_features, out_channels)).to(device)

In [11]:
model

GAE(
  (encoder): GCNEncoder(
    (conv1): GCNConv(3703, 4)
    (conv2): GCNConv(4, 8)
    (conv3): GCNConv(8, 2)
  )
  (decoder): InnerProductDecoder()
)

In [12]:
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

In [13]:
optimizer = Adam(model.parameters(), lr=0.001)

In [14]:
def train():
  model.train()
  optimizer.zero_grad()
  z = model.encode(x, train_pos_edge_index)
  loss = model.recon_loss(z, train_pos_edge_index)
  loss.backward()
  optimizer.step()
  return float(loss)


def test(pos_edge_index, neg_edge_index):
  model.eval()
  with torch.no_grad():
      z = model.encode(x, train_pos_edge_index)
  return model.test(z, pos_edge_index, neg_edge_index)

In [15]:
epochs = 500

for epoch in range(epochs):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    if epoch % 10 == 0:
        print(f'Epoch: {epoch}, Loss: {loss:.4f}, AUC: {auc:.4f}, AP: {ap:.4f}')

Epoch: 0, Loss: 1.3863, AUC: 0.6116, AP: 0.6241
Epoch: 10, Loss: 1.3860, AUC: 0.6584, AP: 0.6928
Epoch: 20, Loss: 1.3848, AUC: 0.6632, AP: 0.6985
Epoch: 30, Loss: 1.3821, AUC: 0.6640, AP: 0.7002
Epoch: 40, Loss: 1.3775, AUC: 0.6657, AP: 0.7026
Epoch: 50, Loss: 1.3716, AUC: 0.6660, AP: 0.7053
Epoch: 60, Loss: 1.3669, AUC: 0.6672, AP: 0.7093
Epoch: 70, Loss: 1.3608, AUC: 0.6685, AP: 0.7143
Epoch: 80, Loss: 1.3540, AUC: 0.6685, AP: 0.7194
Epoch: 90, Loss: 1.3467, AUC: 0.6668, AP: 0.7223
Epoch: 100, Loss: 1.3367, AUC: 0.6650, AP: 0.7247
Epoch: 110, Loss: 1.3265, AUC: 0.6623, AP: 0.7251
Epoch: 120, Loss: 1.3142, AUC: 0.6596, AP: 0.7255
Epoch: 130, Loss: 1.2996, AUC: 0.6587, AP: 0.7252
Epoch: 140, Loss: 1.2880, AUC: 0.6615, AP: 0.7264
Epoch: 150, Loss: 1.2695, AUC: 0.6704, AP: 0.7294
Epoch: 160, Loss: 1.2555, AUC: 0.6966, AP: 0.7387
Epoch: 170, Loss: 1.2344, AUC: 0.7293, AP: 0.7527
Epoch: 180, Loss: 1.2176, AUC: 0.7581, AP: 0.7680
Epoch: 190, Loss: 1.1962, AUC: 0.7766, AP: 0.7796
Epoch: 200,